In [ ]:
import pandas as pd
import numpy as np
import torch
import json

In [ ]:
# All of our fields have names, but we need them in numbers, so define a dictionary to convert.
global magDict
magDict = {
    'TOTUSJH': 0,
    'TOTBSQ': 1,
    'TOTPOT': 2,
    'TOTUSJZ': 3,
    'ABSNJZH': 4,
    'SAVNCPP': 5,
    'USFLUX': 6,
    'TOTFZ': 7,
    'MEANPOT': 8,
    'EPSZ': 9,
    'SHRGT45': 10,
    'MEANSHR': 11,
    'MEANGAM': 12,
    'MEANGBT': 13,
    'MEANGBZ': 14,
    'MEANGBH': 15,
    'MEANJZH': 16,
    'TOTFY': 17,
    'MEANJZD': 18,
    'MEANALP': 19,
    'TOTFX': 20,
    'EPSY': 21,
    'EPSX': 22,
    'R_VALUE': 23,
    'RBZ_VALUE': 24,
    'RBT_VALUE': 25,
    'RBP_VALUE': 26,
    'FDIM': 27,
    'BZ_FDIM': 28,
    'BT_FDIM': 29,
    'BP_FDIM': 30,
    'PIL_LEN': 31,
    'XR_MAX': 32
}

In [ ]:
# Get the data from the JSON file, then return it as a tensor of input data and a list of labels
def getDataFromJSON(path="data/train_partition1_data.json", device='cpu', earlyStop=-1):
    # I might need to refactor these arguments to allow to get the test data. Problem for another day.
    # path is the path to the files, device is where to store it (CUDA), earlyStop is how many lines to 
    # read if you don't want the entire file read.
    
    # Get the dictionary to assign names to numbers
    global magDict
    
    # Get the file and open it. Get it's size. 
    file = open(path)
    lines = file.readlines()
    if earlyStop < 0: length = len(lines)
    else: length = earlyStop
    # This file is rather large, so to not crash my computer we are doing this
    del lines
    file.seek(0)
    
    # Declare a tensor to hold the data, and a list to hold the labels.
    # Dimensions: 0: length of the file. 1: the 33 fields in the data. 2: the 60 observations in each field. 
    tnsr = torch.Tensor().new_empty((length, 33, 60), device=device)
    labels = []
        
    row = -1
    for line in file:
        # Load the line as a dictionary. Row is an integer place and v is a smaller dictionary.
        d: dict = json.loads(line)
        row += 1
        for _, v in d.items(): # we use the _ because we don't want the ID.
            if earlyStop > 0 and row >= earlyStop:
                # If we don't want the entire dataset, stop loading more than we want
                return tnsr, labels
            if row % 100 == 0:
                print(f'Now loading event {row}/{length}')
            # append the label to our list
            labels.append(v['label'])
            
            # Break each individual dictionary into dictionaries of observations
            # Key is the string in magDict, and timeDict is a dictionary of observations over time
            for key, timeDict in v['values'].items():
                # Turn our name string into a numeric value
                location = magDict[key]
                # Get the measurements out of the time series dictionary
                for timeStamp, measurement in timeDict.items():
                    tnsr[int(row)][location][int(timeStamp)] = measurement
    # Close the file. I'm not a heathen                    
    file.close()
    # This might be a good place to perform some post processing, but that's a question for another day. Famous last words.
    return tnsr, labels



In [ ]:
# This file has 77270 data points. 
%time train1, labels1 = getDataFromJSON(path="data/train_partition1_data.json", earlyStop=77270)

In [ ]:
# This file has 93767 data points. 
%time train2, labels2 = getDataFromJSON(path="data/train_partition2_data.json", earlyStop=93767)

In [ ]:
# This file has 42986 data points. 
%time train3, labels3 = getDataFromJSON(path="data/train_partition3_data.json", earlyStop=42986)